In [1]:
from agential.cog.agent.critic import CriticAgent
from agential.cog.agent.critic_old import CriticAgent as OldCriticAgent
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper

from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    CRITIC_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC
)

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

In [2]:
agent = CriticAgent(
    llm=llm, mode="qa", search=search
)

In [3]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

out = agent.generate(
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=CRITIC_INSTRUCTION_HOTPOTQA,
    critique_examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    use_search_tool=True
)

<PROMPT AGENT===========================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

Q: What profession does 

---

In [6]:
from agential.cog.strategies.critic.qa_strategy import QAStrategy

strategy = QAStrategy()

In [8]:
answer = strategy.generate(llm, question, HOTPOTQA_FEWSHOT_EXAMPLES_COT, CRITIC_INSTRUCTION_HOTPOTQA, {})

for idx in range(7):
    critique, critique_additional_keys = strategy.generate_critique(
        llm=llm, 
        question=question, 
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC, 
        answer=answer, 
        prompt=CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA, 
        additional_keys={}, 
        use_interpreter_tool=True
    )
    break

<PROMPT AGENT===========================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

Q: What profession does 

: 